In [3]:
import numpy as np

In [4]:
class OwlOptimization():
    def __init__(self):
        ...
    
    def compile(self,obj,dimension,lb,ub,population_size=100,max_iter=100,optimization="maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    def __initialize_population(self,owl_num,dim,lb,ub):
        return lb+np.random.uniform(0,1,size=(owl_num,dim))*(ub-lb)
    
    def __normalization_intensity(self,fitness_of_owl):
      # Find minimun and maximum value
        b_max = np.amax(fitness_of_owl,axis=-1)[...,np.newaxis]
        w_min = np.amin(fitness_of_owl,axis=-1)[...,np.newaxis]


      # Return normalization intensity of owl position by equation 4
        return (fitness_of_owl - w_min)/(b_max - w_min)
    
    
    def __prey_distance(self,owl_info, prey_info):
        return np.linalg.norm(owl_info - prey_info,axis=-1)
    
    def __change_intensity_owl_loc(self,intensity_owl_loc, distance_info_R):

        return ((intensity_owl_loc)/(distance_info_R**2+0.02)) + np.random.normal(0,1)
    
            
    def __linearly_dec_constant(self,crnt, max_num, range_min=0, range_max=1.9):
        return (crnt-range_min) / (max_num-range_min) * range_max
    
    
    def __update_owl_loc(self,alpha, beta, owl_locations, prey_loc_v, intensity_ICi):

        # Generate random number 0 to 1
        pvm = np.random.uniform(0, 1)

        if pvm < 0.5:
            return owl_locations + beta * intensity_ICi * np.absolute(alpha*prey_loc_v - owl_locations)
        else:
            return owl_locations - beta * intensity_ICi * np.absolute(alpha*prey_loc_v - owl_locations)
        
        
    def calculate_optimal_value(self):
  

        # Get Owl with initial locatins
        owl_loc_with_dim=self.__initialize_population(self.population_size,self.dimension,self.lb,self.ub)

        # Print n * d matrix
        #print(f"Matrix for number of owl with search space:\n\n{owl_loc_with_dim}")


        #Initialize the variable for  fittest candidate
        fittest_owl_V = None
        candidate_evaluations = np.empty([self.population_size])
        # Normalization intensity of owl position
        intensity_owl_loc = self.__normalization_intensity(owl_loc_with_dim)
        for i in range(self.max_iter):

            #Calculating the output of objective function 
            for j in range(self.population_size):
                candidate_evaluations[j]=self.objective_function(owl_loc_with_dim[j,:])

            ## Determining the fittest candidate
            if self.optimization == "maximize":
                fittest_idx = np.argmax(candidate_evaluations)
            elif self.optimization == "minimize":
                fittest_idx = np.argmin(candidate_evaluations)

            ## Updating the fittest candidate
            if fittest_owl_V == None:
                fittest_owl_V = (owl_loc_with_dim[fittest_idx, :], candidate_evaluations[fittest_idx])
            else:
                current_fittest = (owl_loc_with_dim[fittest_idx, :], candidate_evaluations[fittest_idx])

                if self.optimization == "maximize" and fittest_owl_V[1] < current_fittest[1]:
                    fittest_owl_V = current_fittest

                if self.optimization == "minimize" and fittest_owl_V[1] > current_fittest[1]:
                    fittest_owl_V = current_fittest

            # Calculate prey distance
            distance_info_R = self.__prey_distance(owl_loc_with_dim, fittest_owl_V[0])[...,np.newaxis]

            # Now change intensity for ith owl
            intesity_Ici = self.__change_intensity_owl_loc(intensity_owl_loc, distance_info_R)

            alpha = np.random.uniform(0, 0.5)
            beta = self.__linearly_dec_constant((self.population_size-i), self.population_size)
            owl_loc_with_dim = self.__update_owl_loc(alpha, beta, owl_loc_with_dim, fittest_owl_V[0], intesity_Ici)

        return fittest_owl_V


In [6]:
def obj(X):
    sums = 0
    for i in X:
        sums = i*i
    return sums

dimension=3
lb = np.array([-100,-100,-100])
ub = np.array([100,100,100])
max_iter = 200
population_size=100

model = OwlOptimization()
model.compile(obj,dimension,lb,ub,population_size,max_iter,optimization='minimize')
print(model.calculate_optimal_value())

(array([1.22179485e+36, 8.52105810e+36, 1.26442077e-11]), 1.59875988170013e-22)


In [7]:
obj([1.2217e+36,8.5210e+36,1.2644e-11])

1.59870736e-22

0.01
